In [1]:
import pandas as pd
import re
import joblib as pickl


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [4]:
df = pd.read_csv('lenta-ru-news.csv')

/var/folders/1k/rp772snj2y55b6h_cpgh3prh0000gn/T/ipykernel_65764/3394781730.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lenta-ru-news.csv')


In [6]:
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


In [7]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [5]:
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [9]:
df.shape

(494804, 6)

In [10]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [12]:
df[df.text.isna()]['title'].values[0]

'Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента'

In [6]:
df.dropna(subset=['text'],inplace = True)

In [12]:
df.topic.value_counts()

Россия       160442
Мир          136620
Экономика     79528
Спорт         64413
Культура      53796
Name: topic, dtype: int64

In [7]:
RANDOM_STATE = 55

In [8]:
text_transformer = TfidfVectorizer()

In [9]:
%%time
text = text_transformer.fit_transform(df['text'])

CPU times: user 1min 25s, sys: 7.49 s, total: 1min 33s
Wall time: 1min 40s


In [10]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [11]:
%%time
clf = tree.DecisionTreeClassifier()

CPU times: user 464 µs, sys: 5.15 ms, total: 5.62 ms
Wall time: 14.4 ms


In [12]:
clf.fit(X_train,y_train)

In [25]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7931888650503197


In [26]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tatanasidorina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
stop_russian = stopwords.words('russian')

In [29]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [3]:
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
clf2 = GradientBoostingClassifier()


In [2]:
clf2.fit(X_train,y_train)

NameError: name 'clf2' is not defined

In [ ]:
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

In [ ]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

In [ ]:
df['text'] = df['text'].str.replace('/','')

In [ ]:
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [ ]:
df['text_clean'] = remove_trash(df['text'])

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [ ]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic_label'], test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [ ]:
print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

In [ ]:
with open('data/tfidf.pkl', 'wb') as f:
    pickle.dump(clf_norm, f)

with open('data/transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


textA = text_transformer.transform(['бывший тренер сборная россия владимир плющев']   )                              

test = cosine_similarity(text_norm, textA)

In [ ]:
testdf = pd.DataFrame(test)
testdf.columns=['vector']

In [ ]:
testdf.sort_values(by='vector',ascending=False).head(5)

In [ ]:
df[df.index==103437]['text_clean_normal'].values